# Simple Cryptocurrency Proof of Concept in Ruby

In [1]:
require 'openssl'
require 'minitest'
require 'benchmark'
require 'base64'

# Enable assertions outside of Minitest::Test 
include Minitest::Assertions
Object.class_eval do
  attr_accessor :assertions
end
self.assertions = 0

# Override IRuby::Kernal to hide backtrace from display
# https://github.com/SciRuby/iruby/blob/master/lib/iruby/kernel.rb
IRuby::Kernel.class_eval do
  def error_message(e)
    { status: :error,
        ename: e.class.to_s,
        evalue: e.message,
        traceback: ["\e[31m#{e.class}\e[0m: #{e.message}"],
        execution_count: @execution_count }
    end
end


:error_message

## The Miner

The miner takes in a message & finds a `nonce` that satisfies a POW difficulty level

In [2]:
module Miner
  def mine(message, difficulty_level = 2) # 2 leading zeros
    nonce = 0
    loop do
      hash = OpenSSL::Digest::SHA256.new(message + nonce.to_s).to_s
      return [hash, nonce] if hash.start_with? '0' * difficulty_level
      nonce += 1
    end
  end
end

:mine

Time required to mine increases exponentially when difficulty increases. Difficulty would increase according to the moving average of time for each block to be added to the blockchain.

In [3]:
class TestMiner; include Miner; end

puts Benchmark.measure { puts TestMiner.new.mine 'bar' }
puts Benchmark.measure { puts TestMiner.new.mine 'bar', 4 }
# puts Benchmark.measure { puts TestMiner.new.mine 'bar', 6 }

["00b894f575e9311a064511b37bc5bfd57365980ba9157aaf69afec3fecf8178a", 5]
  0.000000   0.000000   0.000000 (  0.000278)

["000065a302ba3d5ff98dd372b82ab1837fbbbae10c4576a9b17d778dfda89955", 61125]
  0.210000   0.000000   0.210000 (  0.209329)



## The Block

A `block` object should have the following information :

- `previous_block` - Previous block on block chain
- `transactions` - Transactions within the block
- `timetamp` - Timestamp of block creation
- `nonce` - Mined nonce
- `hash` - Mined hash

In [4]:
class Block
  include Miner
  
  attr_reader :previous_block, :transactions, :timestamp, :nonce, :hash
  
  def initialize(transactions, previous_block)
    @transactions = transactions
    @timestamp = Time.now
    @previous_block = previous_block
    @hash, @nonce = mine(block_message)
  end
  
  private
  def block_message
    @transactions.to_s + @timestamp.to_s + @previous_block.hash
  end
end

:block_message

## The Genesis Block

Since `blocks` require a `previous_block`, the first `block` will need a dummy GenesisBlock.

In [5]:
class GenesisBlock < Block
  def initialize
    @hash, @nonce = mine(block_message)
  end
  
  def genesis_block
    true
  end
  
  private
  def block_message
    'In the beginning God created the heavens and the earth'
  end
end

:block_message

In [6]:
b0 = GenesisBlock.new
puts [b0.hash, b0.nonce]

b1 = Block.new(['t1', 't2'], b0)
puts [b1.hash, b1.nonce]

["00a6d3080b64cf53c882ede4b5ae71fb3af0d2d1bb573ba2a114c216b675172c", 204]
["00bf9679ad639167c2f04a2a6b31c317d269e01ce98f607dfb6a50b618f95677", 72]


## Wallets

Here we're creating a simple coin wallet with a set of Public/Private keys using a 512 bit RSA for Public Key encryption. 

We'll simply use the an URL safe base64 encoded Public Key as the wallet address.

In [7]:
class Wallet
  attr_reader :address
  
  def initialize
    @key = OpenSSL::PKey::RSA.new(512)  # obviously insecure
    @address = Base64.urlsafe_encode64(@key.public_key.to_der)
  end
  
  def sign(message)
    digest = OpenSSL::Digest::SHA256.new(message)
    sig = @key.sign(digest, message)
    Base64.urlsafe_encode64(sig)
  end
end

w = Wallet.new


#<Wallet:0x007fc8acb15770 @key=#<OpenSSL::PKey::RSA:0x007fc8acb15720>, @address="MFwwDQYJKoZIhvcNAQEBBQADSwAwSAJBANNWPD9DcOayZGfKajEvSiFisJpetK_RHTWPbanTdKm7FlBzOhZUi4tmFtE-WBN6XA_fyebEIhwqMvbohbdRB50CAwEAAQ==">

And then we want to have some helper methods to verify the encryption.

In [8]:
module PKeyVerifier
  def verify(pkey_base64, signature_base64, message)
    pkey = Base64.urlsafe_decode64(pkey_base64)
    signature = Base64.urlsafe_decode64(signature_base64)
    
    key =  OpenSSL::PKey::RSA.new(pkey)
    digest = OpenSSL::Digest::SHA256.new(message)
    key.verify(digest, signature, message)
  end
end

class TestVerifier; include PKeyVerifier; end
pkey_verifier = TestVerifier.new

#<TestVerifier:0x007fc8acafe930>

In [9]:
signature = w.sign('some transaction')
assert pkey_verifier.verify(w.address, signature, 'some transaction')

true

In [10]:
assert pkey_verifier.verify(w.address, signature, 'some altered transaction'), 'Message altered'

Minitest::Assertion: Message altered

In [11]:
another_wallet = Wallet.new
assert pkey_verifier.verify(another_wallet.address, signature, 'some transaction'), 'Message not signed by same wallet'

Minitest::Assertion: Message not signed by same wallet

## Transactions

A transaction would need to have a `source` wallet, and an instructions of amounts to send to a `destination` wallets. 

In [12]:
class Transaction
  attr_reader :source_wallet, :instruction
  
  def initialize(source_wallet, instructions)
    @source_wallet = source_wallet
    @instructions = instructions
  end
end

:initialize

## Transaction Instructions

We'll define a class for Transaction Instructions. Each transaction instruction will define the target wallet & the amount to send.

In [13]:
class Transaction::Instruction
  attr_reader :target_wallet, :amount
  
  def initialize(target_wallet, amount)
    @target_wallet = target_wallet
    @amount = amount.to_f
  end
  
  def to_hash
    { 
      target:  @target_wallet,
      amount: @amount
    }
  end
end

:to_hash

So a typical transaction will look more of less like so 

In [14]:
w1 = Wallet.new
w2 = Wallet.new

t1 = Transaction.new(w1, Transaction::Instruction.new(w2, 25.0))

nil  # hide noisy output

## Transactions List

In a ledger system, all transactions will be a chronologically ordered list. In our case, we'll just add them into a global variable as soon as each transaction is defined.

In [15]:
class Transaction
  attr_reader :source_wallet, :instruction
  
  def initialize(source_wallet, instruction)
    @source_wallet = source_wallet
    @instruction = instruction
    
    append_to_transactions_list
  end
  
  def to_hash
    {
      source: @source_wallet,
      instruction: @instruction.to_hash
    }
  end
  
  def append_to_transactions_list
    $transaction_list << self
  end
end


:append_to_transactions_list

Now, according to the Bitcoin whitepaper, each transaction owner should digitally sign a hash of previous transaction with public key of next owner, so that a payee can verify the chain of ownership.

To achieve that, we'll need to find the last transaction of the `source` wallet, and use it as create a hash & signature. Time to tweak `Transaction`.

In [16]:
class Transaction
  attr_reader :hash, :signature, :previous_hash
  
  def initialize(source_wallet, instruction, hash, signature, previous_hash)
    @source_wallet = source_wallet
    @instruction = instruction
    @hash = hash
    @signature = signature
    @previous_hash = previous_hash
    
    append_to_transactions_list
  end

  def to_hash 
    {
      owner: @source_wallet,
      instruction: @instruction.to_hash,
      hash: @hash,
      signature: @signature,
      previous_hash: @previous_hash,
    }
  end
  
end

:to_hash

In [17]:
class Wallet
  def sign_instruction(instruction)
    previous_hash = last_wallet_transaction && last_wallet_transaction.hash
    hash_payload  = {
      target_address: instruction.target_wallet,
      previous_hash: previous_hash,
      instruction: instruction.to_hash
    }
    hash = OpenSSL::Digest::SHA256.new(hash_payload.to_s)
    signature = sign(hash.to_s)
    
    [hash, signature, previous_hash]
  end
  
  def last_wallet_transaction
    $transaction_list.select {|t| t.source_wallet == self.address}.last
  end
end


:last_wallet_transaction

In [18]:
$transaction_list = []

w1 = Wallet.new
w2 = Wallet.new
w3 = Wallet.new


# We'll pretend that $transaction_list is being transferred through the network as the public ledger
# Since Transactions will be public, all data stored within should be kosher for public viewing (i.e. no private keys)
t1 = Transaction.new(w1.address, i = Transaction::Instruction.new(w2.address, 25.0), *w1.sign_instruction(i))
t2 = Transaction.new(w1.address, i = Transaction::Instruction.new(w2.address, 10.0), *w1.sign_instruction(i))
t3 = Transaction.new(w2.address, i = Transaction::Instruction.new(w3.address, 5.0), *w2.sign_instruction(i))
t4 = Transaction.new(w2.address, i = Transaction::Instruction.new(w3.address, 2.0), *w2.sign_instruction(i))

nil  # hide noisy output

In [19]:
def show_all_transactions
  $transaction_list.map(&:to_hash).each_with_index do |t, index|
    source_address = t[:owner][33..42]  # for display purposes
    target_address = t[:instruction][:target][33..42]  # for display purposes

    puts "Transaction ##{index+1}: \n"
    puts "From: \t\t\t#{source_address}"
    puts "To: \t\t\t\t#{target_address}"
    puts "Amount: \t\t#{t[:instruction][:amount]}"
    puts "Hash: \t\t\t#{t[:hash]}"
    puts "Signature: \t\t#{t[:signature]}"
    puts "Previous Hash: \t#{t[:previous_hash]}"
    puts "----------------------"
  end
end

show_all_transactions

nil  # hide noisy output

Transaction #1: 

From: 			N0Mv0nC-ov
To: 				K8g_UFeYOR
Amount: 		25.0
Hash: 			6fdba30ad3547b2c3f41c1a87b7ac83ae06514fef40d76a5bfa16c2139e255a0
Signature: 		yxqV-scnoW3l9TdgVijV2n9lUqLUruVWTpKUMkCfFQuFYB8XbXr33DzOAWVYI3lCDRO0GJJcy6O-FwlEnq6XEA==
Previous Hash: 	
----------------------
Transaction #2: 

From: 			N0Mv0nC-ov
To: 				K8g_UFeYOR
Amount: 		10.0
Hash: 			e212617fc52b67816ab62332b2e00a774375d59e48d4e3d6062681a4384789b4
Signature: 		Ex9P4gCWDuW8LgfBpUmbnnbEO_Jj1hKlKqgDJWNqqFGggOW-AV7gRnWif1k-yi_oynRsKqi8XTe0VWQRhUAT4w==
Previous Hash: 	6fdba30ad3547b2c3f41c1a87b7ac83ae06514fef40d76a5bfa16c2139e255a0
----------------------
Transaction #3: 

From: 			K8g_UFeYOR
To: 				Mc-OH89Y7w
Amount: 		5.0
Hash: 			52ff1e66c1fafacf9394fc83887b531cd2ee48e0e88e1048e84382edd3570030
Signature: 		gu3kBe46A4m0py638EczwFI1xQbtIJF7WWagCeM876gN8Tr-t9pCiSvGgmIqEOcOX6l0NPR37z2cS33xalB-Eg==
Previous Hash: 	
----------------------
Transaction #4: 

From: 			K8g_UFeYOR
To: 				Mc-OH89Y7w
Amount: 		2.0
Ha

So as the payees, the data integrity can be verified through the signature chains.

In [20]:
class Transaction
  include PKeyVerifier

  def self.find_transaction(hash)
    $transaction_list.find { |t| t.hash == hash }
  end
  
  def verify_all_transactions(transaction_hash = self.hash)
    loop do
      trx = self.class.find_transaction(transaction_hash)
      trx.verify_transaction
      
      break unless @previous_hash
      transaction.verify_all_ancestral_transactions(@previous_hash)
    end
  end
  
  def verify_transaction
    raise 'Invalid transaction' unless verify(@source_wallet, @signature, recalculate_hash.to_s)
  end
  
  def recalculate_hash
    hash_payload = {
      target_address: @instruction.target_wallet,
      previous_hash: previous_transaction && previous_transaction.hash,
      instruction: @instruction.to_hash
    }
    OpenSSL::Digest::SHA256.new(hash_payload.to_s)
  end
  
  def previous_transaction
    self.class.find_transaction(@previous_hash)
  end
end

t1.verify_all_transactions

show_all_transactions

nil  # hide noisy output

Transaction #1: 

From: 			N0Mv0nC-ov
To: 				K8g_UFeYOR
Amount: 		25.0
Hash: 			6fdba30ad3547b2c3f41c1a87b7ac83ae06514fef40d76a5bfa16c2139e255a0
Signature: 		yxqV-scnoW3l9TdgVijV2n9lUqLUruVWTpKUMkCfFQuFYB8XbXr33DzOAWVYI3lCDRO0GJJcy6O-FwlEnq6XEA==
Previous Hash: 	
----------------------
Transaction #2: 

From: 			N0Mv0nC-ov
To: 				K8g_UFeYOR
Amount: 		10.0
Hash: 			e212617fc52b67816ab62332b2e00a774375d59e48d4e3d6062681a4384789b4
Signature: 		Ex9P4gCWDuW8LgfBpUmbnnbEO_Jj1hKlKqgDJWNqqFGggOW-AV7gRnWif1k-yi_oynRsKqi8XTe0VWQRhUAT4w==
Previous Hash: 	6fdba30ad3547b2c3f41c1a87b7ac83ae06514fef40d76a5bfa16c2139e255a0
----------------------
Transaction #3: 

From: 			K8g_UFeYOR
To: 				Mc-OH89Y7w
Amount: 		5.0
Hash: 			52ff1e66c1fafacf9394fc83887b531cd2ee48e0e88e1048e84382edd3570030
Signature: 		gu3kBe46A4m0py638EczwFI1xQbtIJF7WWagCeM876gN8Tr-t9pCiSvGgmIqEOcOX6l0NPR37z2cS33xalB-Eg==
Previous Hash: 	
----------------------
Transaction #4: 

From: 			K8g_UFeYOR
To: 				Mc-OH89Y7w
Amount: 		2.0
Ha

Changes to the transaction instructions or target wallets are quickly detectable throughout the chain.

In [21]:
hacker_wallet = Wallet.new

t2.instance_variable_set(:@instruction, Transaction::Instruction.new(hacker_wallet.address, 10.0))
t2.verify_all_transactions

RuntimeError: Invalid transaction

## Tampering with the public ledger

It is still possible to alter the transactions within the ledger if those Transactions originated from the attacker's wallet, allowing them to reverse any transactions made by them.

In [ ]:
$transaction_list = []

t1 = Transaction.new(w1.address, i = Transaction::Instruction.new(w2.address, 25.0), *w1.sign_instruction(i))
t2 = Transaction.new(w1.address, i = Transaction::Instruction.new(w2.address, 10.0), *w1.sign_instruction(i))
t3 = Transaction.new(w2.address, i = Transaction::Instruction.new(w3.address, 5.0), *w2.sign_instruction(i))
t4 = Transaction.new(w2.address, i = Transaction::Instruction.new(w3.address, 2.0), *w2.sign_instruction(i))

## Why not generate a hash chain using the last transaction?

Need more research on this. Let me know if you know the answer :D

This will just ensure a totally chronological chain of transactions, and the reversal can only be done before another wallet transaction is added to the end. So why not use one single transaction chain?

Maybe, since lots of transactions could be created at any given time, making the last transaction hard to determine. Also it would be too slow for a consensus to made by the entire network to determine which transaction is the last transaction due to the sheer number of transactions made by each node at any given time. 

Maybe that's why transactions are verified by Blocks instead?